# Merging Hospital Pricing Datasets

### Making Sense of Hospital Pricing Transparency

Reference: https://www.cms.gov/hospital-price-transparency

In January 2021, hospitals were required to publish in "a comprehensive machine-readable file with all items and services" that the hospital provides. This project takes two such files - one JSON and one XML - and combines them into a single dictionary which can be accessed programatically, and then pushes the output to a CSV file which is then read back in using Pandas. This was part of an exercise for a programming class in my Master's Degree program in Health Data Science, republished by permission from my instructor.

## Start by accessing and formatting the files

In [3]:
import json
# from lxml import objectify

In [4]:
jsonfile = 'whiteriver.json'
xmlfile = 'saline.xml'

In [5]:
jsondata = json.load(open(jsonfile))

### JSON data is now loaded

### Uncomment the following cell if you haven't installed xmltodict yet

In [6]:
#pip install xmltodict --user  

In [7]:
import xmltodict
f = open(xmlfile)
xmopen = f.read()
f.close()
xmldata = xmltodict.parse(xmopen)
#xmldata

### XML data is now loaded and has been converted to DICT. 

### Store hospital names as variables and define top level data within each structure.

In [8]:
whospital = jsondata['root']['HospitalorFacilityName']

In [9]:
whospital

'White River Medical Center'

In [10]:
wriverdata = jsondata['root']['StandardCharges']

In [11]:
shospital = xmldata['StandardCharges']['Facility']['@Name']

In [12]:
shospital

'SALINE MEMORIAL HOSPITAL'

In [13]:
salinedata = xmldata['StandardCharges']['Facility']['Patient']

### Some data processing and troubleshooting work

Left the following code blocks in so that you could follow my data processing and troubleshooting steps. It turned out the saline.xml data was not formatted consistently, so this required some sleuthing to figure out what was different and where in order to be able to get all of the data added to the dictionary.

In [14]:
#xmldata['StandardCharges']['Facility']['Patient'][1]['Charge']['Item'][1]

In [15]:
#salinedata0 = salinedata[0]  #Need to loop through at this level - LOOP HERE

In [16]:
#salinedata1 = salinedata0['Charge'][0] #Need to loop through at this level - LOOP HERE

In [17]:
#salinedata0['Charge'][0]['Item'][0]['Contracts']['Contract'] #Payor and Price
#salinedata2 = salinedata1['Item'] 

In [18]:
#salinedata2[0] #Individual record starting at 0 - LOOP HERE
#int(salinedata2[0].get('@Code')) #Index to join on = Procedure Code from wrdata
#salinedata2[0].get('Description') #Text Description


#salinedata[0]['Charge'][0]['Item'][0] #formats correctly
#salinedata[0]['Charge']['Item'][0] #error

#salinedata[1]['Charge'][0]['Item'][0] #error
#salinedata[1]['Charge']['Item'][0] #formats correctly

#salinedata[2]['Charge'][0]['Item'][0] #error
#salinedata[2]['Charge']['Item'][0] #formats correctly

#Need to write check for salinedata[x] and run different code based on which one...


In [19]:
#wriverdata[0] #Individual record starting at 0 -- LOOP HERE
#wriverdata[0].get('Description') # Text Description
#int(wriverdata[0].get('ProcedureCode')) # The index to join on

### Define the dictionary file

In [20]:
combodict = {} #{chargecode : , textdescJSON : , {hospital : , {maxcharge : , discountcashcharge : , mincharge : }}}

### Build the initial entries based on the (easier to work with) JSON data

In [21]:
#start building combo with wriver JSON data
#already exist: whospital (name of white river hospital pulled from JSON file)

for x in range(len(wriverdata)):
    
    procedure = int(wriverdata[x].get('ProcedureCode'))
    textdesc = wriverdata[x].get('Description')
    maxcharge = round(float(wriverdata[x].get('MaximumNegotiatedCharge')),2)
    discountcash = round(float(wriverdata[x].get('DiscountedCashPrice')),2)
    mincharge = round(float(wriverdata[x].get('MinimumNegotiatedCharge')),2)
    
    combodict[procedure] = {}
    combodict[procedure][whospital] = {}
    combodict[procedure][whospital]['Charge'] = (maxcharge, discountcash, mincharge)
    combodict[procedure][whospital]['Description'] = textdesc
        
        

### Check the size for comparison later - should be 100

In [23]:
len(combodict)

100

### Add the Saline data from the XML file to the dictionary

In [24]:
#Add Saline data to the dictionary
#shospital defined in previous cell
#The if statement accounts for diff format in salinedata[x]

for a in range(len(salinedata)):
    if a == 0:
        for b in range(len(salinedata[a]['Charge'])):  
            for x in range(len(salinedata[a]['Charge'][b]['Item'])): 

                procedure = salinedata[a]['Charge'][b]['Item'][x].get('@Code')

                if salinedata[a]['Charge'][b]['Item'][x].get('@Code').isdigit():
                    procedure = int(salinedata[a]['Charge'][b]['Item'][x].get('@Code'))


                textdesc = salinedata[a]['Charge'][b]['Item'][x].get('Description')
                maxcharge = round(float(salinedata[a]['Charge'][b]['Item'][x].get('MaxNegotiatedCharge')),2)
                discountcash = round(float(salinedata[a]['Charge'][b]['Item'][x].get('DiscountCashCharge')),2)
                mincharge = round(float(salinedata[a]['Charge'][b]['Item'][x].get('MinNegotiatedCharge')),2)

                if procedure in combodict:
                    combodict[procedure][shospital] = {}
                    combodict[procedure][shospital]['Charge'] = (maxcharge, discountcash, mincharge)
                    combodict[procedure][shospital]['Description'] = textdesc

                else:
                    combodict[procedure] = {}
                    combodict[procedure][shospital] = {}
                    combodict[procedure][shospital]['Charge'] = (maxcharge, discountcash, mincharge)
                    combodict[procedure][shospital]['Description'] = textdesc
                    

    else:
        for x in range(len(salinedata[a]['Charge']['Item'])): 

                procedure = salinedata[a]['Charge']['Item'][x].get('@Code')

                if salinedata[a]['Charge']['Item'][x].get('@Code').isdigit():
                    procedure = int(salinedata[a]['Charge']['Item'][x].get('@Code'))


                textdesc = salinedata[a]['Charge']['Item'][x].get('Description')
                maxcharge = round(float(salinedata[a]['Charge']['Item'][x].get('MaxNegotiatedCharge')),2)
                discountcash = round(float(salinedata[a]['Charge']['Item'][x].get('DiscountCashCharge')),2)
                mincharge = round(float(salinedata[a]['Charge']['Item'][x].get('MinNegotiatedCharge')),2)

                if procedure in combodict:
                    combodict[procedure][shospital] = {}
                    combodict[procedure][shospital]['Charge'] = (maxcharge, discountcash, mincharge)
                    combodict[procedure][shospital]['Description'] = textdesc

                else:
                    combodict[procedure] = {}
                    combodict[procedure][shospital] = {}
                    combodict[procedure][shospital]['Charge'] = (maxcharge, discountcash, mincharge)
                    combodict[procedure][shospital]['Description'] = textdesc


### Check to make sure the data has been added - should exceed 1,500

In [25]:
len(combodict)

1567

### The checkprices() function tests to see if a given procedure code exists - used for troubleshooting

In [26]:
def checkprices(procedurecode):
    """(int OR str) -> values that various hospitals charge for that procedure
    Looks up a dictionary entry for a given procedure code and returns for White River and Saline hospitals
    the max negotiated charge, discount cash price, and min negotiated charge for that procedure.
    
    >>> checkprices(175)
    Code         Hospital            Max   Cash   Min
    175 SALINE MEMORIAL HOSPITAL 11306.76 6689.86 6377.26
    
    >>> checkprices("J2785")
    
    """
    
    if procedurecode in combodict:
        print("Code         Hospital            Max   Cash   Min      Description")
        for hospital in combodict[procedurecode]:
            charges = combodict[procedurecode][hospital]['Charge']
            description = combodict[procedurecode][hospital]['Description']
            print(procedurecode, hospital, charges[0], charges[1], charges[2], description)
    else:
        print("That value is not present")

In [28]:
#checkprices(10120)  #This one has two hospitals data
#checkprices(175)
#checkprices("J2785")
checkprices(99283)   #This is one of the values initially excluded based on the formatting issue

Code         Hospital            Max   Cash   Min      Description
99283 SALINE MEMORIAL HOSPITAL 1284.25 584.83 115.77 EMERGENCY DEPT VISIT


### The printprices() function prints the entire output to the screen - used for troubleshooting

In [29]:
def printprices(dictionary = combodict):
    """ (str(combodict)) -> printed table
    Pulls in an updated dictoary of hospitals, codes, and pricing, etc. and outputs a table with the entire contents
    Defaults to printing combodict as-is
    
    """
    print("Code         Hospital            Max   Cash   Min      Description")
    for procedurecode in dictionary:
        for hospital in combodict[procedurecode]:
            charges = combodict[procedurecode][hospital]['Charge']
            description = combodict[procedurecode][hospital]['Description']
            print(procedurecode, hospital, charges[0], charges[1], charges[2], description)

In [30]:
#printprices()

### The writeprices() function takes in the dictionary that has been built and writes a header plus the formatted output we want into a CSV file

In [31]:
def writeprices(dictionary = combodict):
    """ (str(combodict)) -> printed table
    Pulls in an updated dictoary of hospitals, codes, and pricing, etc. and outputs a table with the entire contents
    Defaults to printing combodict as-is
    
    """
    
    import csv
    
    with open ('prices.csv', 'w', newline='') as csvfile:
        writeme = csv.writer(csvfile)
        writeme.writerow(['Procedure_Code','Hospital','MaxCharge','CashPrice', 'MinCharge', 'Description'])
        
        for procedurecode in dictionary:
            for hospital in combodict[procedurecode]:
                charges = combodict[procedurecode][hospital]['Charge']
                description = combodict[procedurecode][hospital]['Description']
                writeme.writerow([procedurecode, hospital, charges[0], charges[1], charges[2], description])

In [32]:
writeprices()

### Finally, use pandas to read the CSV file into a dataframe to get a nicely formatted version of the data on screen.

In [33]:
import pandas as pd

df = pd.read_csv("prices.csv")

In [34]:
df

,Procedure_Code,Hospital,MaxCharge,CashPrice,MinCharge,Description
0,170,White River Medical Center,8400.99,4677.82,0.00,ANESTH PROCEDURE ON MOUTH
1,220,White River Medical Center,18.48,10.29,0.00,ANESTH INTRCRN NERVE
2,1996,White River Medical Center,224.40,124.95,0.00,HOSP MANAGE CONT DRUG ADMIN
3,10004,White River Medical Center,478.72,266.56,0.00,FNA BX W/O IMG GDN EA ADDL
4,10005,White River Medical Center,2287.12,1273.51,609.74,FNA BX W/US GDN 1ST LES
...,...,...,...,...,...,...
1602,S9126,SALINE MEMORIAL HOSPITAL,174.00,52.20,174.00,"Hospice care, in the home, p"
1603,S9128,SALINE MEMORIAL HOSPITAL,143.00,46.80,143.00,"Speech therapy, in the home,"
1604,S9129,SALINE MEMORIAL HOSPITAL,143.00,46.80,143.00,"Occupational therapy, in the"
1605,S9131,SALINE MEMORIAL HOSPITAL,177.00,53.10,107.00,Pt in the home per diem


### Data has been combined and is now accessible via dictionary or exported CSV version.